## Loading and cleaning the dataset

In [10]:
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import numpy as np
import plotly.express as px

pd.set_option('display.max_colwidth', None)

df_recipes = pd.read_csv('recipes.csv')
df_recipes = df_recipes.drop_duplicates(subset='recipe_id', keep='first')

df_recipes['recipe_id'] = pd.to_numeric(df_recipes['recipe_id'], errors='coerce')
df_recipes.dropna(subset=['recipe_id'], inplace=True)

def remove_text_inside_parentheses(text):
    return re.sub(r'\([^)]*\)', '', text)

## Create column parsed_ingredients
df_recipes['parsed_ingredients'] = pd.Series(dtype='object')
for index, row in df_recipes.iterrows():
    ingredients_list = row['recipe_ingredients'].split('\n')
    cleaned_ingredient_names = set()
    for ingredient in ingredients_list:
        # Remove text inside parentheses
        ingredient = remove_text_inside_parentheses(ingredient)
        # Remove text after "-" and ":" then split by ',' and take the first part
        ingredient_name = ingredient.split('-')[0].split(':')[0].split(',')[0].lower().strip()
        cleaned_ingredient_names.add(ingredient_name)
    df_recipes.at[index, 'parsed_ingredients'] = cleaned_ingredient_names

## Create column num_unique_ingredients to count the number of ingredients for each recipe
df_recipes['num_ingredients'] = df_recipes['parsed_ingredients'].apply(lambda x: len([ingredient for ingredient in x if ingredient]))

df_recipes.describe()

,recipe_id,num_ingredients
count,376.000000,376.000000
mean,9789.132979,7.893617
std,1613.422330,1.696856
min,7165.000000,4.000000
25%,8490.250000,7.000000
50%,9848.000000,8.000000
75%,11067.750000,9.000000
max,12638.000000,14.000000


## Ingredient Frequency Analysis

In [15]:
all_ingredients = []
most_used_ingredients_range = 25

for index, row in df_recipes.iterrows():
    for ingredient in row["parsed_ingredients"]:
        ingredient_name = ingredient.split(',')[0]
        all_ingredients.append(ingredient_name)

ingredient_counts = Counter(all_ingredients)
ingredient_df = pd.DataFrame.from_dict(ingredient_counts, orient='index', columns=['frequency'])
ingredient_df = ingredient_df.sort_values(by='frequency', ascending=False)

most_used_ingredients = ingredient_df.head(most_used_ingredients_range)
fig = px.bar(
    most_used_ingredients, 
    x=most_used_ingredients.index, 
    y='frequency',
    title=f'Top {most_used_ingredients_range} Most Common Ingredients',
    labels={'x': 'Ingredient', 'frequency': 'Frequency'
})

fig.update_layout(xaxis_tickangle=-45, xaxis_title='Ingredient', yaxis_title='Frequency', plot_bgcolor='white')
fig.show()

## Find the X recipes with the least amount of ingredients

In [12]:
sorted_recipes = df_recipes.sort_values(by='num_ingredients', ascending=True)
easiest_recipes_by_ingredients = sorted_recipes.iloc[:2]
easiest_recipes_by_ingredients

,recipe_id,recipe_name,recipe_description,recipe_nutriscore,recipe_ingredients,recipe_reproduction_steps,parsed_ingredients,num_ingredients
329,7881.0,"Filet de saumon, tombée de poireau à la moutarde","15 min de préparation et vous voilà avec un délicieux filet de saumon servi avec du poireau revenu à la moutarde et une purée onctueuse, miam !",A,"Filets de saumon - Frais Embal (240g - 2 personnes), 240 g, ingredient-7881-2-14390\nGousse d'ail, 0.5, ingredient-7881-2-9932\nPoireau, 1, ingredient-7881-2-9710\nPomme de terre, 3, ingredient-7881-2-10029","Step 1: Epluchez et coupez les pommes de terre en morceaux.\r\nDéposez-les dans une casserole et recouvrez-les très largement d'eau. Salez.\r\nPortez à ébullition et faites-les cuire 15 à 20 min.\r\nRéduisez-les en purée. Salez, poivrez.\r\nAjoutez un filet de lait et une noix de beurre pour une purée plus onctueuse.\r\nEn parallèle, préparez la tombée de poireau.\nStep 2: Pressez ou hachez l'ail.\r\nCoupez et retirez la base abîmée du poireau. Incisez-le en deux dans la longueur et rincez-le soigneusement. Emincez-le.\r\nDans une sauteuse, faites chauffer un filet d'huile d'olive à feu moyen à vif.\r\nFaites revenir l'ail et le poireau 10 min environ. Salez, poivrez.\r\nA mi-cuisson, ajoutez un fond d'eau et couvrez pour accélérer la cuisson.\r\nLorsque le poireau est cuit, ajoutez la moutarde, le beurre et mélangez bien. Goûtez et rectifiez l'assaisonnement si nécessaire.\r\nPendant ce temps, faites cuire le saumon.\nStep 3: Rincez, essuyez le filet de saumon puis découpez-le pour obtenir une portion par personne.Salez et poivrez-le.\r\nAstuce : n'hésitez pas à vous aider d'une paire de ciseaux !\r\nDans une poêle, faites chauffer un filet d’huile d’olive à feu moyen à vif.\r\nFaites dorer les pavés de saumon (côté peau en premier) 2 à 3 min de chaque côté jusqu'à ce qu'il soit cuit à coeur. \r\nAstuce : Pour savoir quand retourner le saumon, regardez sur le côté : lorsqu’il est cuit à mi-hauteur, retournez-le !\nStep 4: Servez sans attendre votre filet de saumon accompagné de la purée et de la tombée de poireau à la moutarde !","{gousse d'ail, pomme de terre, poireau, filets de saumon}",4
218,9506.0,Filet de saumon à l’huile de coriandre et patate douce,On prépare une huile à la coriandre maison pour assaisonner ce délicieux filet de saumon ! Accompagné de patates douces pour une note sucrée.,A,"Coriandre, qq brins, ingredient-9506-2-11352\nEndive, 1, ingredient-9506-2-10472\nFilets de saumon - Frais Embal (240g - 2 personnes), 240 g, ingredient-9506-2-14390\nPatate douce, 2, ingredient-9506-2-9930\nPomme elstar, 1, ingredient-9506-2-13046","Step 1: Préchauffez votre four à 220°C en chaleur tournante !\r\nPendant ce temps, épluchez et coupez les patates douces en fines demi-rondelles. \r\nDéposez-les sur une plaque allant au four recouverte de papier cuisson. Versez un filet d'huile d'olive, salez et poivrez. Enfournez 15 min.\r\nEn parallèle, préparez le saumon.\nStep 2: Rincez, essuyez le filet de saumon puis découpez-le pour obtenir une portion par personne. Salez et poivrez-le.\r\nAstuce : n'hésitez pas à vous aider d'une paire de ciseaux !\r\nAu bout des 15 minutes de cuisson des patates douces, ajoutez les filets de saumon sur les patates douces et enfournez de nouveau 10 min.\r\nPendant ce temps, préparez l'huile à la coriandre et la salade. \nStep 3: Ciselez la coriandre (en entier, les tiges se consomment).\r\nDans un bol, mélangez l'huile d'olive et la coriandre. Salez et poivrez. \r\nEn parallèle, dans un saladier, déposez les ingrédients suivant : \r\nVersez l'huile d'olive et le vinaigre. Salez, poivrez et mélangez bien pour obtenir une vinaigrette.\r\nRetirez la base de l'endive. Coupez-la en 2 et retirez la partie centrale dure. Émincez-la.\r\nCoupez la pomme en dés.\nStep 4: Dégustez sans attendre le saumon et les patates douces arrosés d'huile à la coriandre et accompagnés de la salade !","{coriandre, patate douce, pomme elstar, endive, filets de saumon}",5


## Find recipes that contains certain aliments (fish, meat, etc..)

In [13]:
fish_keywords = ['poisson', 'saumon', 'merlu', 'cabillaud', 'truite', 'thon', 'bar', 'maquereau', 'dorade', 'sole', 'églefin', 'sardine', 'morue', 'turbot', 'lieu']
meat_keywords = ['viande', 'poulet', 'boeuf', 'volaille', 'steak', 'entrecôte', 'porc', 'cochon', 'dinde', 'saucisse', 'lapin']

def contains_keywords(ingredients_set, keywords):
    keywords = [keyword.lower() for keyword in keywords]
    return any(keyword in ingredient for ingredient in ingredients_set for keyword in keywords)

df_recipes['contains_fish'] = df_recipes['parsed_ingredients'].apply(lambda x: contains_keywords(x, fish_keywords))
df_recipes['contains_meat'] = df_recipes['parsed_ingredients'].apply(lambda x: contains_keywords(x, meat_keywords))

print("Recipes containing fish:", df_recipes[df_recipes['contains_fish']].shape[0])
print("Recipes containing meat:", df_recipes[df_recipes['contains_meat']].shape[0])

print(df_recipes[df_recipes['contains_fish'] == True][["recipe_id", "recipe_name"]].head(2))

Recipes containing fish: 38
Recipes containing meat: 138
    recipe_id  \
28    12618.0   
46    11862.0   

                                                         recipe_name  
28                        Tartare de saumon fumé, betterave et pomme  
46  Lieu noir meunière à l'orange et risotto d'épeautre à l'estragon  


TODO: try cosine similarity or other method ? Jaccard similarity ?

## Find similar recipes from a "Central" recipe

In [14]:
central_recipe_id = 12047
central_ingredients = df_recipes.loc[df_recipes['recipe_id'] == central_recipe_id, 'parsed_ingredients'].iloc[0]

# Define a function to calculate the number of shared ingredients
def calculate_shared_ingredients(ingredients_set, central_ingredients):
    return len(ingredients_set.intersection(central_ingredients))

# Apply this function across the DataFrame, excluding the central recipe itself
df_recipes['shared_ingredients_count'] = df_recipes.apply(lambda row: calculate_shared_ingredients(row['parsed_ingredients'], central_ingredients) if row['recipe_id'] != central_recipe_id else 0, axis=1)

# Sort the DataFrame based on 'shared_ingredients_count' to find the most similar recipes
similar_recipes = df_recipes[df_recipes['recipe_id'] != central_recipe_id].sort_values(by='shared_ingredients_count', ascending=False)

# Display the top similar recipes, excluding the central one
print(similar_recipes[['recipe_id', 'recipe_name', 'parsed_ingredients', 'shared_ingredients_count']].head())

     recipe_id  \
306     8399.0   
238     9181.0   
47     11874.0   
61     11522.0   
52     12004.0   

                                                                            recipe_name  \
306                                     Tarte fine au curry rouge et salsa de coriandre   
238                                Salade de boulgour, courgette et poulet au gingembre   
47   Curry de céleri-rave au lait de coco et chutney pimenté de courge sucrine du Berry   
61                                      Poulet aux champignons et sabayon à la moutarde   
52                      Paupiette de dinde à la crème de Cognac et purée de céleri-rave   

                                                                                                                                      parsed_ingredients  \
306                             {sucrine, chou rouge, coriandre, fromage blanc, pâte brisée ronde, échalote, sauce curry rouge, mix de haricots égoutté}   
238                             

In [17]:
# Filter recipes that contain "concombre" in their parsed_ingredients
concombre_recipes = df_recipes[df_recipes['parsed_ingredients'].apply(lambda ingredients: 'concombre noa' in ingredients)]

# Display the filtered DataFrame to see the recipes containing "concombre"
print(concombre_recipes[['recipe_id', 'recipe_name', 'parsed_ingredients']])

     recipe_id  \
96     10980.0   
115    10905.0   
117    10698.0   
126    10780.0   
127    10708.0   
128    10696.0   
141    10601.0   
145    10592.0   
149    10563.0   
150    10693.0   
152    10558.0   
159    10466.0   
164    10407.0   
167    10369.0   
169    10522.0   
175     9955.0   
183    10306.0   
204     9917.0   
211     9599.0   
219     9728.0   
226     9708.0   
229     9098.0   
231     9158.0   
233     9205.0   
234     9149.0   
244     9209.0   
249     8975.0   
366     7440.0   
374     7432.0   

                                                                                recipe_name  \
96                     Sabich de patate douce au crémeux d'avocat et pickles d'oignon rouge   
115                               Salade de pastèque au maïs grillé et vinaigrette pimentée   
117                  Salade de concombre/courgette au piment chipotle, yaourt grec et aneth   
126                                           Salade de pastèque grillée, féta 